# 딥러닝 기반 텍스트 파싱을 위한 데이터 레이블링 보고서 (Feat. Doccano & Docker)

이 보고서는 지금까지 제가 혼자 공부한 내용을 단계별로 정리한 내용입니다.

---

### 0. 프로젝트 개요 및 동기

이 프로젝트는 회사에서 할당받은 업무를 효율적으로 해결하기 위해 시작되었습니다. 저에게 주어진 업무는 **각 공공기관 중앙부처의 법령해석 데이터셋을 웹 크롤링하여 PDF 또는 HWP 파일 내의 질의와 회신을 한 세트로 파싱하고 검증**하는 것이었습니다.

구체적으로는 다음과 같은 과제를 안고 있었습니다:

1.  **질의-회신 세트 개수 세기**: 웹 크롤링된 문서에서 '질의 1개'와 '회신 1개'가 정상적으로 하나의 세트를 이루는지 확인하고, 전체 세트의 개수를 파악해야 했습니다.
2.  **질의-회신 연결성 판단**: 질의와 회신 간의 연결이 논리적으로 올바른지, 즉 '질의 1'에 대한 '회신 1'이 제대로 매칭되는지 판단해야 했습니다.
3.  **특이사항 분류**: 특히 질의와 회신의 개수가 맞지 않거나, 회신 내용이 법령에 근거하지 않은 답변일 경우 이를 '특이사항'으로 분류하는 업무가 수반되었습니다.

문제는 이러한 분류 및 검증 작업을 수행해야 할 질의회신 세트가 **총 1만 2천 개**에 달한다는 점이었습니다. 이는 엄청난 **수작업과 시간 소모**를 요구하는 일이었습니다. 컴퓨터 공학 전공자로서 이러한 비효율적인 업무를 제 전공 지식을 활용하여 자동화하고 해결하고자 하는 강한 동기를 느꼈습니다. 단순히 반복 작업을 줄이는 것을 넘어, 이 프로젝트를 통해 **딥러닝, 데이터 라벨링, 도커(Docker), 그리고 자연어 처리(NLP) 개념**을 실질적으로 배우고 적용하는 것을 목표로 삼았습니다. 이 보고서는 이러한 프로젝트에 대한 기록입니다.

---

### 0단계: Google Colab 환경 설정 (프로젝트 시작 전 필수)

본격적인 데이터 레이블링 및 딥러닝 모델 학습에 앞서, Google Colab 환경을 설정하는 것이 중요합니다. 특히 학습에 사용할 GPU/TPU 런타임을 설정하고, Google Drive와 연동하여 데이터를 효율적으로 관리하는 것이 핵심입니다.

#### 0.1. 런타임 유형 설정 (하드웨어 가속기 선택)

딥러닝 모델 학습에는 GPU 또는 TPU와 같은 하드웨어 가속기가 필수적입니다. Colab 노트북을 열면 가장 먼저 이 설정을 확인하고 변경해야 합니다.

1.  Colab 노트북 상단 메뉴에서 **`런타임(Runtime)` > `런타임 유형 변경(Change runtime type)`**을 클릭합니다.
2.  `하드웨어 가속기(Hardware accelerator)` 드롭다운 메뉴에서 **`GPU`** 또는 **`TPU`**를 선택합니다. (일반적으로 `GPU`가 많이 사용되며, `TPU`는 특정 프레임워크에 최적화되어 있습니다.)
3.  `저장(Save)` 버튼을 클릭합니다.

#### 0.2. Google Drive 마운트 (데이터 영속성을 위해)

Colab 런타임은 세션이 종료되면 초기화됩니다. 따라서 데이터나 모델을 영구적으로 저장하기 위해 Google Drive를 마운트해야 합니다. 이를 통해 Doccano에서 내보낸 레이블링 데이터나 학습된 모델을 Drive에 저장하고 로드할 수 있습니다.

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/gdrive')

#### 0.3. 프로젝트 파일 구조 설정 (Google Drive 내)

Google Drive 내에 일관된 프로젝트 파일 구조를 생성하여 데이터를 체계적으로 관리합니다. 저는 `Colab Notebooks` 폴더 아래에 `deep-learning` 프로젝트 폴더를 만들고, 그 안에 `data`와 같은 서브 폴더를 만들었습니다. 이는 Doccano에서 내보낼 레이블링 데이터를 저장할 위치가 됩니다.

In [ ]:
import os
# Google Drive 내 프로젝트 폴더 경로 설정 (당신의 Drive 구조에 맞게 수정하세요)
project_root = '/content/gdrive/MyDrive/Colab Notebooks/deep-learning/'
data_dir = os.path.join(project_root, 'data')
model_dir = os.path.join(project_root, 'model')

# 필요한 폴더 생성 (이미 있다면 넘어감)
os.makedirs(data_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

print(f"프로젝트 루트: {project_root}")
print(f"데이터 폴더: {data_dir}")
print(f"모델 폴더: {model_dir}")

#### 0.4. 필요한 라이브러리 설치

Hugging Face `transformers`와 `datasets` 라이브러리는 딥러닝 모델 학습 및 데이터 처리에 필수적입니다. Colab 런타임이 초기화될 때마다 설치해 주어야 합니다.

In [ ]:
# Hugging Face Transformers 및 Datasets 라이브러리 설치
!pip install transformers datasets accelerate

---

### 1단계: 프로젝트 목표 설정 및 딥러닝 기반 파싱 이해

제 목표는 특정 텍스트 파일(소방시설 법령 질의회신집)에서 **'질의 N - 회신 N' 세트**를 정확하게 추출하는 것이었습니다. 단순히 정규 표현식으로 패턴을 찾는 것에는 한계가 있다고 판단했습니다. 특히 질의와 회신 사이에 불필요한 텍스트(페이지 번호, 다른 법령 내용 등)가 끼어 있는 복잡한 상황을 해결하려면, **딥러닝 모델, 그중에서도 특정 엔티티를 인식하는 NER(Named Entity Recognition) 기술이 적합하다는 것을 알게 되었습니다.**

여기서 중요한 점은 **지도 학습(Supervised Learning) 기반 딥러닝 모델은 반드시 '레이블링된 데이터'가 필요하다는 것**이었습니다. 모델에게 "이건 질의야", "저건 회신이야"라고 명확하게 가르쳐줄 정답지가 있어야만 학습할 수 있기 때문입니다.

---

### 2단계: 효율적인 레이블링 도구 선택의 중요성

처음에는 Colab에서 파이썬 코드로 직접 레이블링 데이터를 만들거나, 텍스트 편집기에서 수동으로 `<TAG>내용</TAG>`을 입력하는 방식을 고려했습니다. 하지만 6천 줄이 넘는 방대한 데이터를 수작업으로 처리하는 것은 **비효율적이고 오류 발생 가능성이 매우 높다**는 것을 깨달았습니다.

그래서 **전문적인 텍스트 어노테이션 툴**을 사용하기로 결정했습니다. 여러 툴 중 **Doccano**가 오픈소스이면서 NER에 특화된 기능을 제공하여 제 프로젝트에 가장 적합하다고 판단했습니다.

하지만 Doccano를 Colab에서 직접 실행하는 것은 세션 유지의 어려움, 환경 설정의 복잡성, 그리고 안정성 문제 때문에 바람직하지 않다는 것을 배웠습니다. 가장 효율적인 방법은 **로컬 PC에 Doccano를 설치하고 사용하는 것**이었고, 이때 **Docker**가 가장 쉽고 안정적인 설치/실행 방법을 제공한다는 것을 알게 되었습니다.

---

### 3단계: Docker 설치 및 기본 환경 설정

Doccano를 사용하기 위해 가장 먼저 해야 할 일은 제 Windows PC에 Docker를 설치하는 것이었습니다.

#### 3.1. Docker Desktop 다운로드 및 설치

1.  [Docker 공식 웹사이트](https://www.docker.com/products/docker-desktop/)에 접속했습니다.
2.  제 PC의 프로세서가 **Intel 또는 AMD 기반(x64)**임을 확인했습니다. (Windows 설정 > 시스템 > 정보에서 '시스템 종류'가 'x64 기반 프로세서'임을 확인했습니다.)
3.  Windows용 Docker Desktop 설치 파일을 다운로드했습니다.
4.  다운로드한 설치 파일을 실행하여 Docker Desktop을 설치했습니다. 설치 과정 중 **WSL 2 (Windows Subsystem for Linux 2) 활성화**에 대한 안내가 나왔고, 지시에 따라 이를 활성화했습니다. (WSL 2는 Docker 컨테이너의 성능과 안정성을 위해 필수적이라는 것을 알게 되었습니다.)
5.  설치가 완료된 후 PC를 재시작했습니다.

#### 3.2. Docker 설치 확인

터미널(PowerShell)을 열어 Docker가 제대로 설치되었는지 확인했습니다.

In [ ]:
# NOTE: 이 명령어는 Colab이 아닌 당신의 로컬 PC 터미널(PowerShell/CMD)에서 실행해야 합니다.
docker --version

In [ ]:
# NOTE: 이 명령어는 Colab이 아닌 당신의 로컬 PC 터미널(PowerShell/CMD)에서 실행해야 합니다.
docker run hello-world

---

### 4단계: Doccano Docker 컨테이너 실행 및 초기 설정

이제 Docker가 준비되었으니 Doccano를 실행할 차례였습니다.

#### 4.1. Doccano 컨테이너 실행

이전에 시도했을 때 "Conflict. The container name /doccano is already in use" 오류가 발생하여 기존 컨테이너를 삭제하고 다시 시도했습니다. 이 오류는 컨테이너가 이미 존재하지만 실행 중이지 않을 때 발생한다는 것을 배웠습니다. 또한, `Exited(1)` 오류 로그를 통해 Doccano 컨테이너 시작 시 필요한 관리자 계정 환경 변수(`ADMIN_USERNAME`, `ADMIN_EMAIL`, `ADMIN_PASSWORD`)가 누락되었다는 것을 확인했습니다.

그래서 기존 컨테이너를 삭제하고, 필요한 환경 변수를 포함하여 새로운 컨테이너를 실행했습니다. 이 계정은 Docker Hub 계정과는 완전히 별개이며, Doccano 애플리케이션에 로그인할 때 사용할 계정입니다.

In [ ]:
# NOTE: 이 명령어는 Colab이 아닌 당신의 로컬 PC 터미널(PowerShell/CMD)에서 실행해야 합니다.

# 1. 기존 컨테이너 삭제 (혹시 이전 시도에서 Exited 상태로 남아있다면)
docker rm doccano

# 2. 새로운 Doccano 컨테이너를 관리자 계정 환경 변수와 함께 실행
# 중요한: your_admin_username, your_admin_email@example.com, your_admin_password는 실제 사용할 정보로 변경해야 합니다!
docker run -it -d --name doccano -p 8000:8000 \
-e ADMIN_USERNAME=my_doccano_admin \
-e ADMIN_EMAIL=my.email@example.com \
-e ADMIN_PASSWORD=my_secure_password \
doccano/doccano

#### 4.2. 컨테이너 실행 확인

컨테이너가 정상적으로 `Running` 상태로 실행 중인지 확인했습니다.

In [ ]:
# NOTE: 이 명령어는 Colab이 아닌 당신의 로컬 PC 터미널(PowerShell/CMD)에서 실행해야 합니다.
docker ps

---

### 5단계: Doccano 웹 인터페이스 접속 및 프로젝트 설정

이제 모든 준비가 완료되었고, Doccano 웹 인터페이스에 접속하여 레이블링 프로젝트를 설정했습니다.

#### 5.1. Doccano 접속 및 로그인

웹 브라우저를 열고 `http://localhost:8000`으로 접속했습니다. 이전 단계에서 `-e` 옵션으로 설정한 관리자 계정(`my_doccano_admin`, `my_secure_password`)으로 로그인했습니다.

#### 5.2. 새 프로젝트 생성

1.  로그인 후 대시보드에서 **`Create new project`**를 클릭했습니다.
2.  `Project Name`은 `소방시설법령_질의회신_NER`로 설정했습니다.
3.  `Project Type`은 텍스트 내 엔티티를 추출하는 작업이므로 **`Sequence Labeling`**을 선택했습니다. **여기서 'Sequence Labeling'은 사실상 'NER(Named Entity Recognition)'과 동일한 개념이라는 것을 알게 되었습니다.** 텍스트의 연속된 시퀀스(단어 또는 구)에서 특정 의미를 가진 엔티티를 식별하고 레이블을 붙이는 작업이기 때문입니다. Doccano가 내부적으로 BIO(Beginning, Inside, Outside) 태깅 방식을 사용하여 데이터를 처리해 줄 것이므로, 저는 엔티티명만 정의하면 됩니다.
4.  `Save`를 클릭하여 프로젝트를 생성했습니다.

#### 5.3. 레이블(Labels) 정의

왼쪽 사이드바에서 **`Labels`** 메뉴로 이동하여, 제가 추출하고자 하는 패턴에 대한 레이블들을 하나씩 추가했습니다.

* **`QUESTION_ID`**: '질의 1', '질의 2'와 같은 질의 번호
* **`QUESTION_CONTENT`**: 질의의 실제 내용
* **`ANSWER_ID`**: '회신 1', '회신 2'와 같은 회신 번호
* **`ANSWER_CONTENT`**: 회신의 실제 내용
* **`MISC_HEADER`**: 질의/회신 사이에 끼어 있는 문서 제목, 섹션 제목 등 불필요한 헤더 정보
* **`LAW_CONTENT`**: 질의/회신 중간에 인용된 법령 본문

각 레이블에 단축키와 색상을 지정하여 레이블링의 효율을 높일 준비를 마쳤습니다.

#### 5.4. 데이터 가져오기 (Import Data)

마지막으로, 레이블링할 원본 텍스트 파일을 Doccano로 가져왔습니다.

1.  왼쪽 사이드바에서 **`Import Data`**를 클릭했습니다.
2.  `Select file` 버튼을 눌러 제 PC에 저장된 `promptech1.txt` 파일을 선택했습니다.
3.  `File format`은 **`Plain Text`**로 설정하고 `Upload`를 클릭했습니다. Doccano가 각 줄을 개별 문서로 가져오는 것을 확인했습니다.

---

### 6단계: 텍스트에 레이블 달기 (Annotate) - 현재 진행 중

현재 저는 Doccano의 **`Annotate`** 섹션에서 텍스트에 직접 레이블을 달고 있습니다.

* 화면에 표시되는 텍스트를 마우스로 드래그하여 원하는 부분을 선택합니다.
* 선택 후, 왼쪽에 정의된 레이블(예: `QUESTION_CONTENT`)을 클릭하거나 설정한 단축키를 누릅니다.
* 선택된 텍스트 위에 레이블이 적용된 것을 확인합니다.
* `QUESTION_ID`, `QUESTION_CONTENT`, `ANSWER_ID`, `ANSWER_CONTENT` 등 주요 패턴을 중점적으로 레이블링하고 있습니다.
* 질의/회신 사이에 끼어 있는 불필요한 텍스트(예: 페이지 번호, 다른 섹션 제목)는 `MISC_HEADER`나 `LAW_CONTENT`로 레이블링하거나, 혹은 **아무것도 선택하지 않고 넘어가면 `O` (Other) 태그로 처리됩니다.**
* 주기적으로 `Save` 버튼을 눌러 작업 내용을 저장하고, `Next` 버튼으로 다음 문서로 이동하며 레이블링을 진행하고 있습니다.

---

### 7단계: 레이블링된 데이터 내보내기 및 Colab에서 로드 (향후 계획)

레이블링 작업이 완료되면, Doccano에서 **`Export Data`** 기능을 사용하여 레이블링된 데이터를 **`JSONL (JSON Lines)`** 형식으로 내보낼 것입니다. 이 파일을 Google Drive에 업로드한 후, Colab 노트북에서 로드하여 딥러닝 모델 학습을 본격적으로 시작할 계획입니다.

다음은 Colab에서 Doccano로부터 내보낸 JSONL 파일을 로드하고 Hugging Face Dataset 형식으로 변환하는 예시 코드입니다. 이 코드는 레이블링 작업 완료 후 Colab 노트북에서 직접 실행할 것입니다.

In [ ]:
# NOTE: 이 코드는 Doccano에서 레이블링 및 Export Data까지 완료된 후, Colab에서 실행합니다.

import os
import json
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# 1. Doccano에서 정의한 레이블 목록 (B-, I- 없이)
doccano_raw_labels = ["QUESTION_ID", "QUESTION_CONTENT", "ANSWER_ID", "ANSWER_CONTENT", "MISC_HEADER", "LAW_CONTENT"]

# 2. 모델 학습을 위한 최종 BIO 레이블 목록 및 매핑
label_list = ["O"] # 'Other' 태그는 항상 포함
for label in doccano_raw_labels:
    label_list.append(f"B-{label}") # Beginning 태그
    label_list.append(f"I-{label}") # Inside 태그

label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}
num_labels = len(label_list)

print(f"모델 학습을 위한 최종 레이블 목록: {label_list}")

# 3. Doccano에서 내보낸 JSONL 파일 경로 설정 (Google Drive에 업로드된 파일)
# 이 경로는 0.3단계에서 설정한 data_dir과 일치해야 합니다.
project_root = '/content/gdrive/MyDrive/Colab Notebooks/deep-learning/'
labeled_data_file_path = os.path.join(project_root, 'data', 'your_doccano_export.jsonl') # 'your_doccano_export.jsonl'를 실제 파일명으로 변경!

# 4. 사용할 토크나이저 (promptech1.txt가 한국어이므로 한국어 모델 추천)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# 5. Doccano JSONL 로드 및 Hugging Face Dataset 형식으로 변환
converted_data_for_hf = []
try:
    with open(labeled_data_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            doc_data = json.loads(line)
            text = doc_data['text']
            annotations = doc_data.get('label', []) # Doccano의 'label' 필드: [[start_offset, end_offset, "LABEL_NAME"], ...]

            # 텍스트를 토큰화하고 각 토큰의 원본 텍스트에서의 위치(offset)를 함께 가져옵니다.
            # max_length는 모델의 최대 입력 길이를 고려하여 조정합니다. (예: BERT 계열은 512)
            tokenized_output = tokenizer(text, return_offsets_mapping=True, truncation=True, max_length=512)
            input_ids = tokenized_output['input_ids']
            offsets = tokenized_output['offset_mapping']
            # tokens = tokenizer.convert_ids_to_tokens(input_ids) # 디버깅용 토큰 (필요시 주석 해제)

            # 각 토큰에 해당하는 BIO 레이블을 초기화
            labels = [-100] * len(input_ids) # -100은 손실 계산에서 무시될 특수 토큰 또는 서브워드 토큰의 나머지 부분

            word_ids = tokenized_output.word_ids(batch_index=0)
            previous_word_idx = None
            
            # 토큰별로 레이블 할당 (Doccano의 스팬 기반 레이블을 토큰 기반 BIO 레이블로 변환)
            annotation_idx = 0 # 현재 처리할 어노테이션의 인덱스
            for token_idx, word_idx in enumerate(word_ids):
                if word_idx is None: # CLS, SEP와 같은 특수 토큰 건너뛰기
                    labels[token_idx] = -100
                elif word_idx != previous_word_idx: # 새로운 단어의 첫 번째 서브워드 토큰
                    current_word_start_offset = offsets[token_idx][0]
                    current_word_end_offset = offsets[token_idx][1]
                    
                    assigned_label = "O"
                    # 현재 토큰의 단어 오프셋에 해당하는 어노테이션 찾기
                    for ann_start, ann_end, ann_label_name in annotations:
                        # 단어의 시작 오프셋이 어노테이션 범위 내에 있고, 끝 오프셋도 범위 내에 있는 경우 (단어가 완전히 어노테이션에 포함)
                        if ann_start <= current_word_start_offset and current_word_end_offset <= ann_end:
                            if ann_start == current_word_start_offset: # 어노테이션의 시작과 단어의 시작이 일치
                                assigned_label = f"B-{ann_label_name}"
                            else: # 어노테이션 내부에 있지만 시작은 아님
                                assigned_label = f"I-{ann_label_name}"
                            break # 해당 어노테이션을 찾았으니 다음 토큰으로
                    labels[token_idx] = label_to_id[assigned_label]
                else: # 같은 단어의 후속 서브워드 토큰 (I- 태그)
                    # 이전 토큰의 레이블이 B- 또는 I- 였다면, 이 토큰은 동일한 I- 태그를 부여합니다.
                    if labels[token_idx-1] != -100 and id_to_label[labels[token_idx-1]].startswith(('B-', 'I-')):
                         labels[token_idx] = label_to_id[id_to_label[labels[token_idx-1]].replace('B-', 'I-')]
                    else: # 그 외의 경우 (예: 이전 토큰이 O 또는 -100), 현재 토큰도 -100
                         labels[token_idx] = -100

                previous_word_idx = word_idx

            # 실제 모델 입력에 필요한 형태로 저장
            converted_data_for_hf.append({
                "input_ids": input_ids,
                "attention_mask": tokenized_output['attention_mask'],
                "labels": labels # 이 labels는 ID로 변환된 BIO 태그 리스트
            })

    # Hugging Face Dataset으로 변환
    processed_df = pd.DataFrame(converted_data_for_hf)
    hf_dataset = Dataset.from_pandas(processed_df)

    print(f"Hugging Face Dataset으로 변환된 샘플 수: {len(hf_dataset)}")
    print("\n변환된 Hugging Face Dataset 첫 번째 샘플:")
    print(hf_dataset[0])
    print(f"디코딩된 토큰: {tokenizer.convert_ids_to_tokens(hf_dataset[0]['input_ids'])}")
    print(f"디코딩된 레이블: {[id_to_label[l_id] if l_id != -100 else 'O' for l_id in hf_dataset[0]['labels']]}")

    # 이제 hf_dataset을 학습, 검증, 테스트 세트로 분할하여 모델 학습에 사용할 수 있습니다.
    # train_test_split_dataset = hf_dataset.train_test_split(test_size=0.2, seed=42)
    # train_dataset = train_test_split_dataset["train"]
    # eval_dataset = train_test_split_dataset["test"]

except FileNotFoundError:
    print(f"오류: 레이블링된 파일 '{labeled_data_file_path}'을(를) 찾을 수 없습니다. Google Drive에 업로드했는지 확인하세요.")
except Exception as e:
    print(f"레이블링된 파일을 로드하거나 처리하는 중 오류 발생: {e}")

---

지금까지의 과정은 딥러닝 프로젝트의 성공적인 시작을 위한 필수적인 데이터 준비 단계였습니다. 시간과 노력이 많이 들어가지만, 정확하고 일관된 레이블링 데이터는 모델 성능에 직접적인 영향을 미치므로 매우 중요합니다.